# Script to help user to define a configuration cell for ODC notebooks

*****

Geographical extent, time period, measurements, Coordinate Reference System (CRS) and resolution differ between the datasets available on the SDC.

This script will help you to create a configuration cell to load the data that you want, to be manually copy/pasted or loaded in ODC Jupyter notebook.


In [1]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../sdc-notebooks/Tools/define_mod_locs.py

import pyproj

import numpy as np
import ipywidgets as widgets

from shapely.geometry import Polygon
from dea_tools.datahandling import mostcommon_crs

from odc.ui import DcViewer
from pyproj import Proj, transform

from utils.data_cube_utilities.dc_display_map import _degree_to_zoom_level

from sdc_tools.sdc_utilities import new_get_query_metadata, get_native_epsg_and_res, humanize_measure
from sdc_tools.sdc_display_map import draw_map

import datacube
dc = datacube.Datacube()

# silence warning (not recommended during development)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Allow user to select an available product prioritizing ingested (if any)over indexed

product_names = []

products = dc.list_products()
prfxs = set(prod.split('_')[0] for prod in products['name'])
for prfx in prfxs:
    for prprods in [[x for x in products['name'] if x.startswith(prfx)]]:
        # print(prprods)
        if len(prprods) == 1:
            product_names.append(prprods[0])
        else:
            swissprod = [x for x in prprods if x.endswith('_swiss')]
            if len(swissprod) == 1:
                product_names.append(swissprod[0])
            else:
                product_names.extend(prprods)
product_names.sort()
                
# Select the product
product_sel = widgets.RadioButtons(options=product_names, disabled=False)
display(widgets.Label('Select a product and run the next cell: '), product_sel)

Label(value='Select a product and run the next cell: ')

RadioButtons(options=('GE1', 'RapidMapping', 'arealstatistik', 'astgtmv003', 'corinelc_europe', 'corinelc_swit…

In [3]:
# Get product metadata and select measurements

mtd = new_get_query_metadata(dc, product_sel.value)
# if too slow use the line below (will not be accurate with all product !)
# mtd = new_get_query_metadata(dc, product_sel.value, quick = True)

full_lat = mtd['lat_extents']
full_lon = mtd['lon_extents']
min_max_dates = mtd['time_extents']
def_crs = mtd['crs']

measurements = dc.list_measurements()
measurements_for_product = filter(lambda x: x['product'] == product_sel.value, measurements)
df = measurements.loc[product_sel.value, ['name', 'aliases']]

try:
    df['tmp'] = [', '.join(map(str, l)) for l in df['aliases']]
    df['all'] = df['name'] + ': ' + df['tmp']
except:
    df['all'] = df['name']

measur_sel = widgets.SelectMultiple(options=sorted(list(df['all'])),
                                        disabled=False)
display(widgets.Label('Select measurements (displayed with their aliases) and run the next cell: '), measur_sel)


The product don't have a defined CRS and resolution, then geographical extents are estimated in epsg:4326 !



Label(value='Select measurements (displayed with their aliases) and run the next cell: ')

SelectMultiple(options=('QA_PIXEL: pq, pixel_quality', 'QA_RADSAT: radsat, radiometric_saturation', 'SR_B1: ba…

In [4]:
# Convert selection to measurements list and message 

humanize = True # Use human understandable measurement alias

measur_list = list(measur_sel.value)

measur_sel = []
msg = '# to make your live easier you can manually replace the measurements variable by \n' \
      '# one of their alias:\n'
for m in measur_list:
    if humanize:
        measur_sel.append(humanize_measure(m))
    else:
        measur_sel.append(m.split(': ')[0])
    msg = f"{msg}# {m}\n"

In [5]:
# Explore with DcViewer

lats = full_lat
lons = full_lon

if not def_crs is None:
    x0, y0 = transform(Proj(init = def_crs), Proj(init = 'epsg:4326'),lons[0], lats[0])
    x1, y1 = transform(Proj(init = def_crs), Proj(init = 'epsg:4326'),lons[1], lats[1])
    lats = (y0, y1)
    lons = (x0, x1)

ctr = (np.mean(lats), np.mean(lons))

# Calculate zoom level based on coordinates
margin = -0.5
zoom_bias = 0
lat_zoom_level = _degree_to_zoom_level(margin = margin, *lats ) + zoom_bias
lon_zoom_level = _degree_to_zoom_level(margin = margin, *lons) + zoom_bias
zoom = min(lat_zoom_level, lon_zoom_level)

DcViewer(dc = dc,
         products = [product_sel.value],
         time = str(min_max_dates[1].year), 
         width = "100%",
         center = ctr,
         zoom = zoom,
         max_datasets_to_display = 3000,
         style = {'weight': 1, 'fillOpacity': 0.1})

In [6]:
# Select geographical extent by adding an empty map you can draw on it

crs = def_crs
if crs is None:
    crs = 'epsg:4326'

m, poly = draw_map(lon_ext = full_lon, lat_ext = full_lat, crs = crs, src = True, fsc = True)
print('Zoom, pan and draw a rectangle or polygon (the bounding box will be used) and run the next cell:')
m

Zoom, pan and draw a rectangle or polygon (the bounding box will be used) and run the next cell:


Map(center=[46.735, 8.495000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…

In [7]:
# Once a feature is drawn, extract the bounding box of the last feature drawn

coords = poly.last_draw['geometry']['coordinates']
geo_extent = Polygon(coords[0]).bounds

min_lon = geo_extent[0]
min_lat = geo_extent[1]
max_lon = geo_extent[2]
max_lat = geo_extent[3]

In [8]:
# Get CRS and resolution

if not def_crs is None:
    epsg = def_crs.split(':')[1]
    res_x = mtd['lon_res']
    res_y = mtd['lat_res']
else:
    # get native epsg code and resolution of a given measurement (of a given product)
    nat_epsg, res_x, res_y = get_native_epsg_and_res(dc, product_sel.value, measur_sel[0])

    # get most common CRS within the defined parameters
    crs_wkt = mostcommon_crs(dc = dc, product = product_sel.value,
                             query = {'measurements': measur_sel,
                                      'longitude': (min_lon, max_lon),
                                      'latitude': (min_lat, max_lat),
                                      'time': min_max_dates})
    epsg = pyproj.CRS(crs_wkt).to_epsg()

    if nat_epsg != epsg:
        print(f"! the native and most commong epsgs differ ({nat_epsg}, {epsg} respectively) !\n! You might need to adapt the `output_crs`and `resolution`parameters !!!")

In [9]:
crs_wkt

'PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",9],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32632"]]'

In [10]:
# Select time period

start_date = widgets.DatePicker(description='Start date',
                                value = min_max_dates[0].date(),
                                disabled=False)
end_date = widgets.DatePicker(description='End date',
                              value = min_max_dates[1].date(),
                              disabled=False)
display(widgets.Label('IF REQUIRED define time period (cannot be outside of the initial displayed time) and run the next cell:'),
        widgets.HBox([start_date, end_date]))

Label(value='IF REQUIRED define time period (cannot be outside of the initial displayed time) and run the next…

In [11]:
# Check defined time period

assert start_date.value >= min_max_dates[0].date(), \
       'Start date cannot be defined before {}'.format(min_max_dates[0].date())
assert end_date.value <= min_max_dates[1].date(), \
       'End date cannot be defined after {}'.format(min_max_dates[1].date())
assert start_date.value <= end_date.value, \
       'End date is defined before start date'

# end_date = end_date.value + timedelta(days=1) # end_date is not inclusive !

print('Time period is OK')

Time period is OK


In [12]:
# Resume configuration parameters in a format ready to be copy/pasted to a new cell,
# and in a txt file to be loaded with the '%load config_cell.txt' magic.

str = f'''# Configuration

product = '{product_sel.value}'
measurements = {measur_sel}
{msg}
longitude = ({min_lon}, {max_lon})
latitude = ({min_lat}, {max_lat})
crs = 'epsg:4326'

time = ('{start_date.value.strftime('%Y-%m-%d')}', '{end_date.value.strftime('%Y-%m-%d')}')
# the following date formats are also valid:
# time = ('2000-01-01', '2001-12-31')
# time=('2000-01', '2001-12')
# time=('2000', '2001')

output_crs = 'epsg:{epsg}'
resolution = -{res_y}, {res_x}'''

print(str)
with open('config_cell.txt', 'w') as text_file:
    print(str, file=text_file)

# Configuration

product = 'landsat_ot_c2_l2'
measurements = ['QA_PIXEL', 'red']
# to make your live easier you can manually replace the measurements variable by 
# one of their alias:
# QA_PIXEL: pq, pixel_quality
# SR_B4: band_4, red

longitude = (7.089272, 7.102146)
latitude = (46.828638, 46.83862)
crs = 'epsg:4326'

time = ('2013-03-22', '2023-08-22')
# the following date formats are also valid:
# time = ('2000-01-01', '2001-12-31')
# time=('2000-01', '2001-12')
# time=('2000', '2001')

output_crs = 'epsg:32632'
resolution = -30.0, 30.0
